# Gemini API with images and video input

In [1]:
!pip install google-cloud-aiplatform
!pip install rich

In [2]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

import google.generativeai as genai
genai.configure(api_key=GOOGLE_API_KEY)

from rich.console import Console
console = Console()

Gemini 1.5 Pro and 1.5 Flash support a maximum of 3,600 image files.

Images must be in one of the following image data MIME types:
*   PNG - image/png
*   JPEG - image/jpeg
*   WEBP - image/webp
*   HEIC - image/heic
*   HEIF - image/heif

Each image is equivalent to 258 tokens.

For best results:

*   Rotate images to the correct orientation before uploading.
*   Avoid blurry images.
*   If using a single image, place the text prompt after the image.

In [3]:
!curl -o animales_mexico.jpeg https://storage.googleapis.com/questionsanswersproject/animales_mexico.jpeg

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  283k  100  283k    0     0  1358k      0 --:--:-- --:--:-- --:--:-- 1360k


In [4]:
ref_image = genai.upload_file(path="animales_mexico.jpeg", display_name="animales_mexico")
console.print(f"{ref_image=}")

ref_image=genai.File({
    'name': 'files/u78d7x3n1sa6',
    'display_name': 'animales_mexico',
    'mime_type': 'image/jpeg',
    'sha256_hash': 'YTc0NTkyNTVmY2M1OTY0Njk4NTkwYjgxMGI3NmMxZTAzNTA3NWQ3OTA2MTNhNDA3NTlhNTQ0ZWIyZGZiYjQ1ZA==',
    'size_bytes': '289881',
    'state': 'ACTIVE',
    'uri': 'https://generativelanguage.googleapis.com/v1beta/files/u78d7x3n1sa6',
    'create_time': '2024-07-26T04:02:03.189971Z',
    'expiration_time': '2024-07-28T04:02:03.178460021Z',
    'update_time': '2024-07-26T04:02:03.189971Z'})

In [5]:
file = genai.get_file(name=ref_image.name)
console.print(f"{file=}")

file=genai.File({
    'name': 'files/u78d7x3n1sa6',
    'display_name': 'animales_mexico',
    'mime_type': 'image/jpeg',
    'sha256_hash': 'YTc0NTkyNTVmY2M1OTY0Njk4NTkwYjgxMGI3NmMxZTAzNTA3NWQ3OTA2MTNhNDA3NTlhNTQ0ZWIyZGZiYjQ1ZA==',
    'size_bytes': '289881',
    'state': 'ACTIVE',
    'uri': 'https://generativelanguage.googleapis.com/v1beta/files/u78d7x3n1sa6',
    'create_time': '2024-07-26T04:02:03.189971Z',
    'expiration_time': '2024-07-28T04:02:03.178460021Z',
    'update_time': '2024-07-26T04:02:03.189971Z'})

In [6]:
model = genai.GenerativeModel('gemini-1.5-flash-001')
prompt = [ref_image, "Write a nature blog with the following mexican animals:"]
response = model.generate_content(prompt, stream=True)
for chunk in response:
  console.print(chunk.text)

##

A Journey Through Mexico's Diverse Wildlife: From Axolotls to Prairie

Dogs

Mexico is a land of vibrant colors, rich history, and incredibly diverse

wildlife. Today, we embark on a journey through its fascinating ecosystems, encountering some of the most unique 
creatures that call this land home.

Our first stop takes

us to the heart of Mexico City, where the **axolotl** thrives in the Xochimilco canals. This extraordinary 
amphibian, resembling a tiny

...

further, we find ourselves in the arid plains of northern Mexico, where the **prairie dog** reigns supreme. These 
social creatures, known for their high-pitched barks and intricate burrow systems, are a vital part of their 
ecosystem.

Their tunnels help aerate the soil and create habitat for other species, highlighting their vital role in 
maintaining the delicate balance of the landscape.

Next, we journey to the dry, open landscapes of Mexico's central plateau, home to the elusive **black-tailed 
jackrabbit**. This graceful creature, with its long ears

and powerful legs, is a master of speed and agility, able to leap incredible distances as it escapes predators. 
Their remarkable adaptations to the harsh environment make them a symbol of resilience in the face of adversity.

Finally, we venture into the rugged Sierra Madre mountains, where the **Mexican wolf** roams free.

This endangered species, once widespread throughout the region, now faces the challenges of habitat loss and human 
encroachment.  Their majestic presence and vital role as apex predators serve as a reminder of the 
interconnectedness of all living things.

From the fascinating axolotl to the agile black-tailed jackrabbit, Mexico's diverse

wildlife paints a vivid picture of the country's unique beauty and ecological importance. Each creature, from the 
smallest amphibian to the largest mammal, plays a vital role in the intricate web of life that sustains this 
vibrant land.

In [7]:
prompt = "Return a bounding box for the Axolotl. \n [ymin, xmin, ymax, xmax]"
response = model.generate_content([ref_image, prompt])

print(response.text)

The bounding box for the Axolotl is [61, 126, 412, 432].


# Prompt with multiple images

In [8]:
!curl -o got.jpg https://storage.googleapis.com/questionsanswersproject/got.jpg
!curl -o trump.jpeg https://storage.googleapis.com/questionsanswersproject/trump.jpeg

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 67883  100 67883    0     0   368k      0 --:--:-- --:--:-- --:--:--  370k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  157k  100  157k    0     0  1240k      0 --:--:-- --:--:-- --:--:-- 1246k


In [12]:
model = genai.GenerativeModel('gemini-1.5-flash-001')
ref_image_got = genai.upload_file(path="got.jpg", display_name="got")
ref_image_trump = genai.upload_file(path="trump.jpeg", display_name="trump")

prompt = ["""
      image 1 <ref_image_got>
      image 2 <ref_image_trump>
      Create a funny story where combine images 1 and 2
    """,
    ref_image_got,
    ref_image_trump]

response = model.generate_content(prompt)
console.print(response.text)

The Night King, ruler of the White Walkers and the Iron Throne, was facing a crisis. His armies were depleted, his 
resources dwindling, and his popularity was at an all-time low.  He needed a new strategy, a way to rally his 
troops and inspire fear in the hearts of his enemies.

Then, a rogue raven arrived, bearing a strange, alien message: "Make America Great Again." The message was 
accompanied by a grainy image of a man with an orange-hued complexion, a furrowed brow, and a strange affinity for 
hairspray. 

Intrigued, the Night King called for his advisors. "This man," he rasped, "this... Trump, claims to be a master of 
strategy. He has a vast network of followers, a loyal army, and a knack for manipulating the masses." 

The advisors, having never heard of "strategy" or "manipulating the masses," exchanged nervous glances. 

"Perhaps," said the Night King, "we can learn from this Trump. He is said to be a master of "winning." Maybe, just 
maybe, he can teach us how to win the Great War."

Thus began an unlikely alliance. The Night King, a fearsome warrior of ice and darkness, sent his most trusted 
lieutenant, the Night King's right-hand man, a pale figure with a melancholic air and a strong desire to be known 
as "the one who finally killed that annoying red-headed guy," to meet with Trump. 

The lieutenant, who had been told to call himself "the King's Shadow," arrived at Trump Tower, unsure what to 
expect. He was greeted by a swarm of suits, a cacophony of voices, and a potent odor of hairspray.  

After a brief meeting, the King's Shadow found himself in a room with Trump, staring at a giant TV screen. Trump, 
gesturing wildly, was ranting about a trade deal with the dragons. 

"You need to get tough," Trump declared, "build a wall, a big, beautiful wall! And, most importantly, make sure you
win! You can't lose!  You have to be the best!"

The King's Shadow, accustomed to silent commands and strategies based on brute force, was utterly bewildered. But 
he listened intently, absorbing Trump's unorthodox teachings. 

Trump, meanwhile, was oblivious to the King's Shadow's true nature. He saw him as a potential ally, a powerful 
force he could use to further his own agenda. 

And so, the Night King, under the guidance of Trump's strategic brilliance, began to transform his army. They 
adopted Trump's slogans, replacing the chilling roar of the White Walkers with a chant of "Make Winter Great 
Again."  They started building a wall, a massive, ice-covered barrier designed to keep the living out. And most 
importantly, they learned to embrace the power of propaganda, spreading fear and disinformation with unprecedented 
speed. 

The living, bewildered by the White Walkers' newfound fervor, began to doubt their own strength.  They had been so 
focused on defeating the Night King, they had forgotten how to win.

In the end, the Night King, armed with Trump's unique brand of "strategy," was unstoppable.  He crushed the living,
built a wall around the whole world, and declared himself the rightful ruler of winter, an icy tyrant with a deep 
understanding of the power of a good slogan.  The Night King, with a smirk of triumph, sat on his throne, the iron 
throne, and proclaimed: "Winter is here, and it's going to be great again!"

# Prompting with video

Video must be in one of the following video format MIME types:

*  video/mp4
*  video/mpeg
*  video/mov
*  video/avi
*  video/x-flv
*  video/mpg
*  video/webm
*  video/wmv
*  video/3gpp

The File API service extracts image frames from videos at 1 frame per second (FPS) and audio at 1Kbps, single channel, adding timestamps every second

Individual frames are 258 tokens, and audio is 32 tokens per second

In [13]:
!wget https://storage.googleapis.com/questionsanswersproject/simpsons-first-episode.mp4

--2024-07-26 04:04:53--  https://storage.googleapis.com/questionsanswersproject/simpsons-first-episode.mp4
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.181.207, 173.194.206.207, 173.194.193.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.181.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4205052 (4.0M) [video/mp4]
Saving to: ‘simpsons-first-episode.mp4’

simpsons-first-epis 100%[===================>]   4.01M  --.-KB/s    in 0.05s   

2024-07-26 04:04:53 (75.4 MB/s) - ‘simpsons-first-episode.mp4’ saved [4205052/4205052]



In [14]:
import time

ref_video = genai.upload_file(path="simpsons-first-episode.mp4", display_name="simpsons-first-episode")

# Check whether the file is ready to be used.
while ref_video.state.name == "PROCESSING":
    print('.', end='')
    time.sleep(1)
    ref_video = genai.get_file(ref_video.name)

if ref_video.state.name == "FAILED":
  raise ValueError(ref_video.state.name)

console.print(f"{ref_video=}")

..

ref_video=genai.File({
    'name': 'files/9iz811oiopiu',
    'display_name': 'simpsons-first-episode',
    'mime_type': 'video/mp4',
    'sha256_hash': 'M2ZhN2VmOWQyMmU4MDA1M2ZiOTcxZTIxMjhmYTg0NDZhNTVlZTg1YmY0OGZkZWIyNWNjYWY5OTUwNzU0NmM0Nw==',
    'size_bytes': '4205052',
    'state': 'ACTIVE',
    'uri': 'https://generativelanguage.googleapis.com/v1beta/files/9iz811oiopiu',
    'video_metadata': {'video_duration': '108s'},
    'create_time': '2024-07-26T04:04:57.835273Z',
    'expiration_time': '2024-07-28T04:04:57.829061186Z',
    'update_time': '2024-07-26T04:05:00.254851Z'})

In [15]:
prompt = "Identify the characters in the video"
model = genai.GenerativeModel('gemini-1.5-flash-001')
response = model.generate_content([ref_video, prompt])
console.print(response.text)

The characters in the video are Homer Simpson, Marge Simpson, Bart Simpson, and Maggie Simpson.

In [17]:
prompt = "What are between 01:05 and 01:19"
model = genai.GenerativeModel(model_name="gemini-1.5-pro")
response = model.generate_content([ref_video, prompt],
                                  request_options={"timeout": 600})
print(response.text)

The scene is dark between 01:05 and 01:09, and there are four pairs of eyes blinking in the dark between 01:14 and 01:19. 


In [19]:
prompt = "Transcribe video from 00:05 until 00:30"
model = genai.GenerativeModel(model_name="gemini-1.5-pro")
response = model.generate_content([ref_video, prompt],
                                  request_options={"timeout": 600})
print(response.text)

0:05 Um, Dad? Yeah?
0:06 What is the mind? Is it just a system of impulses or is it something tangible?
0:13 Relax.
0:14 What is mind? No matter.
0:17 What is matter? Never mind.
0:20 Thanks, Dad.
0:22 Good night, son.
0:26 Good night, Lisa.
0:28 Good night, Mom.
0:29 Sweet dreams.
0:31 Thanks, Mom. Sleep tight.
0:34 I will, Mom. Don't let the bedbugs bite.
0:40 Bedbugs?
0:43 Rock a bye baby, in the tree top.
0:48 When the wind blows, the cradle will rock.
0:55 When the bough breaks, the cradle will fall.
1:01 And down will come baby cradle and all.
1:09 Sweet dreams. 

